## Food Cutting OWLReady Integration Notebook

OWLReady Integration with owlready2 for the Food Cutting SPARQL Queries from: https://github.com/Food-Ninja/CuttingFood/blob/main/jupyter/FoodCuttingQueries.ipynb

In [ ]:
from symbol import return_stmt

import owlready2
from owlready2 import get_ontology, Restriction, And

- The URL can be a file from the internet but also a local file
- Defining the Namespaces globally so they can be accessed by every function
- Accessing a class requires to call the Namespace for example, the class "apple" (IRI: "http://purl.obolibrary.org/obo/FOODON_03301710") corresponds to OBO.FOODON_03301710, as the OBO namespace is defined as such.

In [ ]:
url = "https://raw.githubusercontent.com/Food-Ninja/CuttingFood/main/owl/food_cutting.owl"
onto = get_ontology(url).load()
SOMA = onto.get_namespace("http://www.ease-crc.org/ont/SOMA.owl#")
CUT2 = onto.get_namespace("http://www.ease-crc.org/ont/situation_awareness#")
CUT = onto.get_namespace("http://www.ease-crc.org/ont/food_cutting#")
DUL = onto.get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = onto.get_namespace("http://purl.obolibrary.org/obo/")


- Utility function to retrieve all restrictions, the restrictions are structured as parent classes of the food object.
- requires food object class as input, for example: OBO.FOODON_03301710

In [ ]:
def get_restrictions_of_class(cls):
    union = set()
    for parent in cls.is_a:
        if not isinstance(parent, owlready2.Restriction):
            for e in parent.is_a:
                if isinstance(e, owlready2.Restriction):
                    union.add(e)
        else:
            union.add(parent)
    
    return list(union)

- Query for the required Tool, requires a food object as input.
- The food object class has a restriction which shows which CuttingTool is required.
- First we consider only the restrictions with the property: "SOMA.hasDisposition" and check if the food object can be cut.
- Secondly we check if a CuttingTask can be afforded, by looking at the property "SOMA.affordsTask".
- Finally we can retrieve the required tool with the "SOMA.affordsTrigger" property.

In [ ]:
def gettool_query(food_object=None):
    restrictions = get_restrictions_of_class(food_object)
    for restriction in restrictions:
        if isinstance(restriction, owlready2.Restriction) and restriction.property == SOMA.hasDisposition:
            for r in restriction.value.Classes:
                if r == CUT2.Cuttability:
                    for c in restriction.value.Classes:
                        if isinstance(c, owlready2.Restriction) and c.property == SOMA.affordsTask:
                            if c.value != CUT.CuttingTask:
                                return "No CuttingTask found"
                            continue
                        if isinstance(c, owlready2.Restriction) and c.property == SOMA.affordsTrigger:
                            return c.value.value

- This function checks if and which prior actions are required, for example if the core of a fruit needs to be removed.
- First we want to check if an action is required by looking at the "CUT.hasPart" property, this property contains classes like "Cut.ShouldBeAvoided" or "MustBeAvoided", which indicates that a prior action is required.
- If the case occurs, we look at the Class which is retrieved from the "CUT.hasPart" property, like "Stem" and "Core". This classes also contain restrictions from which we can retrieve the needed action, by asking the property "SOMA.affordsTask".
- The output can be for example "StemRemoving" or "CoreCutting"

In [ ]:
def needed_action_query(food_object=None):
    needed_actions = []
    restrictions = get_restrictions_of_class(food_object)
    for i in restrictions:
        if isinstance(i, owlready2.Restriction) and i.property == CUT.hasPart:
            for c in i.value.Classes:
                if isinstance(c, owlready2.Restriction) and c.property == CUT.hasEdibility:
                    if c.value == CUT.ShouldBeAvoided or c.value == CUT.MustBeAvoided:
                        for j in i.value.Classes:
                            if not isinstance(j, owlready2.Restriction):
                                if len(get_restrictions_of_class(j)) == 0:
                                    continue
                                if len(get_restrictions_of_class(j)) > 0:
                                    for h in get_restrictions_of_class(j):
                                        for k in h.value.Classes:
                                            if not isinstance(k, owlready2.Restriction):
                                                continue
                                            if isinstance(k, owlready2.Restriction) and k.property == SOMA.affordsTask:
                                                needed_actions.append(k.value)


    return needed_actions
    

- Returns the prior task, if required, of a given task, by accessing the "CUT.requiresPriorTask" property.

In [ ]:
def get_prior_task(task=None):
    restrictions = [i for i in task.is_a if isinstance(i, owlready2.Restriction)]
    if len(restrictions) == 0:
        if task.is_a:
            return get_prior_task(task.is_a[0])
    if len(restrictions) < 0:
        for i in task.is_a:
            if isinstance(i, owlready2.Restriction) and i.property == CUT.requiresPriorTask:
                return i.value

- Returns the number of repetitions required for a given task, by accessing the "CUT.repetitions" property.

In [ ]:
def get_number_of_repetitions(task=None):
    restrictions = [i for i in task.is_a if isinstance(i, owlready2.Restriction)]
    if len(restrictions) == 0:
        if task.is_a:
            return get_number_of_repetitions(task.is_a[0])

    if len(restrictions) > 0:
        for i in task.is_a:
            if isinstance(i, owlready2.Restriction) and i.property == CUT.repetitions:
                if i.cardinality:
                    return "min repetitions:" + str(i.cardinality)
                else:
                    return i.value




print(get_number_of_repetitions(SOMA.Dicing))

- Returns the position of execution, by accessing the "CUT.affordsPosition" property.

In [ ]:
def get_position_of_execution(task=None):
    restrictions = [i for i in task.is_a if isinstance(i, owlready2.Restriction)]
    if len(restrictions) == 0:
        if task.is_a:
            return get_position_of_execution(task.is_a[0])

    if len(restrictions) > 0:
        for i in restrictions:
            if i.property == CUT.affordsPosition:
                return i.value

print(get_position_of_execution(CUT.Snipping))

## Test

- Testing the Queries for every task and every FoodObject

In [ ]:
tasks = [('Cutting Action',"cut:CuttingAction"),
        ('Quartering', "cut:Quartering"),
        ('Julienning',"cut:Julienning"),
        ('Halving',"cut:Halving"),
        ('Dicing',"soma:Dicing"),
        ('Cutting',"soma:Cutting"),
        ('Slicing',"soma:Slicing"),
        ('Snipping',"cut:Snipping"),
        ('Slivering',"cut:Slivering"),
        ('Sawing',"cut:Sawing"),
        ('Paring',"cut:Paring"),
        ('Carving',"cut:Carving"),
        ('Mincing',"cut:Mincing"),
        ('Cubing',"cut:Cubing"),
        ('Chopping',"cut:Chopping")]

objects=[('almond', "obo:FOODON_00003523"),
        ('apple', "obo:FOODON_03301710"),
        ('avocado', "obo:FOODON_00003600"),
        ('banana', "obo:FOODON_00004183"),
        ('bean', "obo:FOODON_03301403"),      
        ('citron', "obo:FOODON_03306596"),
        ('coconut', "obo:FOODON_00003449"),     
        ('cucumber', "obo:FOODON_00003415"),
        ('kiwi', "obo:FOODON_00004387"), 
        ('kumquat', "obo:FOODON_03306597"),
        ('lemon', "obo:FOODON_03301441"),
        ('lime', "obo:FOODON_00003661"),
        ('olive', "obo:FOODON_03317509"),
        ('orange', "obo:FOODON_03309832"),
        ('peach', "obo:FOODON_03315502"), 
        ('pepper', "obo:FOODON_00003520"),
        ('pineapple', "obo:FOODON_00003459"),
        ('pumpkin', "obo:FOODON_00003486"),
        ('strawberry', "obo:FOODON_00003443"),        
        ('squash', "obo:FOODON_00003539"),
        ('tomato', "obo:FOODON_03309927")]

In [ ]:
task = SOMA.Slicing

print(get_number_of_repetitions(task))
print(get_position_of_execution(task))
print(get_prior_task(task))

In [ ]:
for o in objects:
    foodOn_cls = getattr(OBO,o[1].split(":")[1])
    print(foodOn_cls)
    print(foodOn_cls.label)
    print(gettool_query(foodOn_cls))
    print(needed_action_query(foodOn_cls))
    print("New Cls")


- Anscheinend werden nur die Sachen berücksichtigt wo ShouldbeAvoided oder MustBeAvoided drin steht, vlt das ändern?
- Manchmal sind mehrere Actions als Rückgabe zb Apple hat Core das removed wird, daraus
- Manche Klassen geben nichts aus bei needed actions, obwohl es drin stehen sollte, auch bei den queries wird nichts returned. Meistens bei "Shell"
- affordsTask Peeling wird nicht berücksichtigt bei WhatTool?